# Introduction
Here we explore training of neural networks that predict the GERP score from the sequence. We try multiple network architectures and report the results of each.


Inversion symmetry of score implies local window should extend both sides equally - since if one sequence affects only in one direction, its reverse complement will affect in the opposite direction

## Dependencies

In [13]:
import logging
import dill
import os
import numpy as np
from data.load import read_sequence, read_annotation_generator, examine_annotation, read_gerp_scorer
from data.paths import chr17_paths # paths to source data files
from data.process import get_train_test_x_y

# from score_nn_modeling import LocalWindowModel, LocalTransformerEncoderModel, ModelTrainer
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
# from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
# from lightgbm import LGBMRegressor

from score_nn_modeling import LocalWindowModel, SymmetricLocalWindowModel, LocalTransformerEncoderModel, ModelTrainer

import jax
import jax.numpy as jnp
import pandas as pd
import optax

In [2]:
# random seed
SEED = 200

# display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 3)

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data

In [4]:
# start the analysis with human chromosome 17
paths = chr17_paths

# get the raw sequence dictionary, from a FASTA file
seq_dict = read_sequence(paths.sequence)

# # (optional) examine annotations, from the annotation GFF file
# examine_annotation(paths.annotation)

# get the annotated sequence generator function, from the annotation GFF file
seq_records_gen = read_annotation_generator(paths.annotation, seq_dict=seq_dict)

# get GERP retrieval function, from the BigWig file
gerp_scorer = read_gerp_scorer(paths.gerp)

In [6]:
# One hot encode the sequence in a sliding window
# process from scratch, or load dill / pickle file if done before
xy_datapath = './__state__/xy_data.pkl'
if os.path.isfile(xy_datapath):
    logging.info(f'Loading saved xy data from {xy_datapath}.')
    with open(xy_datapath, 'rb') as file:
        xy_data = dill.load(file)
        x_train, y_train, x_test, y_test = xy_data
else:
    x_train, y_train, x_test, y_test = get_train_test_x_y(seq_records_gen, gerp_scorer, ['CDS'], max_train_rows=2000000)
    
    logging.info(f'Saving xy data to {xy_datapath}.')
    with open(xy_datapath, 'wb') as file:
        xy_data = (x_train, y_train, x_test, y_test)
        dill.dump(xy_data, file)
    

INFO:root:Loading saved xy data from ./__state__/xy_data.pkl.


# Classic ML Regression Models
## Model selection
Here we train different ML models, and rate them with train and test R^2, which measures the proportion of the variance in the scores we are trying to predict that is explained by the model.

In [12]:
regressor_classes = [XGBRegressor, GradientBoostingRegressor, ElasticNet, SGDRegressor, BayesianRidge, LinearRegression]

In [13]:
ml_models = {}
for reg_class in regressor_classes:
    model = reg_class()
    model.fit(x_train, y_train.ravel())
    
    class_name = reg_class.__name__
    train_R2 = model.score(x_train, y_train.ravel())
    test_R2 = model.score(x_test, y_test.ravel())
    logging.info(f'Regressor: {class_name}.  Train R^2: {train_R2:.3f}.  Test R^2: {test_R2:.3f}.')
    
    ml_models[class_name] = model

INFO:root:Regressor: XGBRegressor.  Train R^2: 0.126.  Test R^2: 0.118.
INFO:root:Regressor: GradientBoostingRegressor.  Train R^2: 0.069.  Test R^2: 0.071.
INFO:root:Regressor: ElasticNet.  Train R^2: 0.000.  Test R^2: -0.000.
INFO:root:Regressor: SGDRegressor.  Train R^2: 0.051.  Test R^2: 0.053.
INFO:root:Regressor: BayesianRidge.  Train R^2: 0.052.  Test R^2: 0.054.
INFO:root:Regressor: LinearRegression.  Train R^2: 0.052.  Test R^2: 0.054.


## Narrowing the dependence
Given we conclude XGBRegressor is the best performing, we can use it with different local window size and offsets. We do this by trimming the input features by various values.

In [16]:
nbases = 4
width = x_train.shape[1]//nbases
train_R2_all, test_R2_all = np.zeros((width, width)), np.zeros((width, width))
for start in range(width):
    for end in range(start, width):
        
        x_train_sub = x_train[:,start*nbases:(end+1)*nbases]
        x_test_sub = x_test[:,start*nbases:(end+1)*nbases]
        
        logging.info(f'Start {start}, End {end}, x_train_sub.shape {x_train_sub.shape}')
        
        model = XGBRegressor()
        model.fit(x_train_sub, y_train.ravel())
        
        train_R2 = model.score(x_train_sub, y_train.ravel())
        test_R2 = model.score(x_test_sub, y_test.ravel())
        
        train_R2_all[start, end] = train_R2
        test_R2_all[start, end] = test_R2
        
        logging.info(f'\tTrain R^2: {train_R2:.3f}.  Test R^2: {test_R2:.3f}.')

INFO:root:Start 0, End 0, x_train_sub.shape (1525923, 4)
INFO:root:	Train R^2: 0.008.  Test R^2: 0.009.
INFO:root:Start 0, End 1, x_train_sub.shape (1525923, 8)
INFO:root:	Train R^2: 0.010.  Test R^2: 0.011.
INFO:root:Start 0, End 2, x_train_sub.shape (1525923, 12)
INFO:root:	Train R^2: 0.013.  Test R^2: 0.014.
INFO:root:Start 0, End 3, x_train_sub.shape (1525923, 16)
INFO:root:	Train R^2: 0.023.  Test R^2: 0.024.
INFO:root:Start 0, End 4, x_train_sub.shape (1525923, 20)
INFO:root:	Train R^2: 0.026.  Test R^2: 0.026.
INFO:root:Start 0, End 5, x_train_sub.shape (1525923, 24)
INFO:root:	Train R^2: 0.031.  Test R^2: 0.029.
INFO:root:Start 0, End 6, x_train_sub.shape (1525923, 28)
INFO:root:	Train R^2: 0.040.  Test R^2: 0.039.
INFO:root:Start 0, End 7, x_train_sub.shape (1525923, 32)
INFO:root:	Train R^2: 0.044.  Test R^2: 0.042.
INFO:root:Start 0, End 8, x_train_sub.shape (1525923, 36)
INFO:root:	Train R^2: 0.058.  Test R^2: 0.054.
INFO:root:Start 0, End 9, x_train_sub.shape (1525923, 40)

In [20]:
print(100*np.round(train_R2_all, 5))
print(100*np.round(test_R2_all,5))

[[ 0.795  0.967  1.291  2.262  2.589  3.075  4.032  4.424  5.831  7.24
   8.905  9.645 10.408 10.689 11.013 11.639 11.779 12.099 12.553]
 [ 0.     0.091  0.328  1.351  1.615  2.079  3.167  3.586  4.997  6.411
   8.075  8.874  9.791 10.073 10.427 11.073 11.232 11.563 12.092]
 [ 0.     0.     0.052  0.98   1.208  1.605  2.685  3.101  4.518  5.932
   7.612  8.461  9.416  9.69  10.111 10.791 10.946 11.258 11.823]
 [ 0.     0.     0.     0.812  0.998  1.357  2.369  2.724  4.134  5.543
   7.251  8.144  9.119  9.436  9.856 10.542 10.77  11.093 11.626]
 [ 0.     0.     0.     0.     0.088  0.352  1.415  1.713  3.022  4.439
   6.167  7.123  8.245  8.604  9.08   9.871 10.085 10.447 11.054]
 [ 0.     0.     0.     0.     0.     0.069  1.038  1.292  2.499  3.828
   5.547  6.525  7.756  8.165  8.663  9.508  9.725 10.083 10.703]
 [ 0.     0.     0.     0.     0.     0.     0.819  1.005  2.144  3.331
   4.97   5.985  7.289  7.724  8.285  9.159  9.421  9.786 10.413]
 [ 0.     0.     0.     0.     0.  

# Neural Network Models
Here we train different neural network architectures, and rate them with the mean square error loss as well as R^2.

## Local Window Fully Connected Model

In [14]:
model1 = LocalWindowModel()

In [15]:
model_trainer1 = ModelTrainer(model1, epochs=20000, optimizer=optax.sgd)

In [16]:
model_trainer1.train(x_train, y_train)

INFO:root:Epoch 0. Training loss: 7.3023. R^2: -0.0302.
INFO:root:Epoch 100. Training loss: 6.9039. R^2: 0.0260.
INFO:root:Epoch 200. Training loss: 6.6829. R^2: 0.0571.
INFO:root:Epoch 300. Training loss: 6.6343. R^2: 0.0640.
INFO:root:Epoch 400. Training loss: 6.6011. R^2: 0.0687.
INFO:root:Epoch 500. Training loss: 6.5675. R^2: 0.0734.
INFO:root:Epoch 600. Training loss: 6.5299. R^2: 0.0787.
INFO:root:Epoch 700. Training loss: 6.4990. R^2: 0.0831.
INFO:root:Epoch 800. Training loss: 6.4764. R^2: 0.0863.
INFO:root:Epoch 900. Training loss: 6.4587. R^2: 0.0888.
INFO:root:Epoch 1,000. Training loss: 6.4449. R^2: 0.0907.
INFO:root:Epoch 1,100. Training loss: 6.4333. R^2: 0.0924.
INFO:root:Epoch 1,200. Training loss: 6.4233. R^2: 0.0938.
INFO:root:Epoch 1,300. Training loss: 6.4143. R^2: 0.0950.
INFO:root:Epoch 1,400. Training loss: 6.4065. R^2: 0.0961.
INFO:root:Epoch 1,500. Training loss: 6.3992. R^2: 0.0972.
INFO:root:Epoch 1,600. Training loss: 6.3931. R^2: 0.0980.
INFO:root:Epoch 1,

In [18]:
loss_train = model_trainer1.model.loss(x_train, y_train)
r2_score_train = 1 - train_loss/jnp.var(y_train)

loss_test = model_trainer1.model.loss(x_test, y_test)
r2_score_test = 1 - test_loss/jnp.var(y_test)

logging.info(f'Train loss {loss_train:.3f}. R^2: {r2_score_train:.3f}.')
logging.info(f'Test loss {loss_test:.3f}. R^2: {r2_score_test:.3f}.')

INFO:root:Train loss 6.255. R^2: 0.118.
INFO:root:Test loss 5.924. R^2: 0.112.


In [39]:
with open('./__state__/local_model_trainer.pkl', 'wb') as file:
    dill.dump(model_trainer1, file)

## Symmetric Local Window Fully Connected Model
Similar to the past model, but where the symmetry of GERP score with respect to a sequence and its reverse complement is hardcoded in the sequence structure.

It seems to perform a little worse than the unrestricted local window model above. This makes sense, since it is a constrained version of the former. This difference can be used to separate the sequence dependent variation that is strand independent (i.e. symmetric) from the sequence dependent variation that is strand dependent (i.e. not symmetric). Though a more rigorous way of handling this is needed.

In [26]:
model2 = SymmetricLocalWindowModel()

In [27]:
model_trainer2 = ModelTrainer(model2, epochs=20000, optimizer=optax.sgd)

In [28]:
model_trainer2.train(x_train, y_train)

INFO:root:Epoch 0. Training loss: 7.3228. R^2: -0.0331.
INFO:root:Epoch 100. Training loss: 6.7698. R^2: 0.0449.
INFO:root:Epoch 200. Training loss: 6.6704. R^2: 0.0589.
INFO:root:Epoch 300. Training loss: 6.6389. R^2: 0.0634.
INFO:root:Epoch 400. Training loss: 6.6195. R^2: 0.0661.
INFO:root:Epoch 500. Training loss: 6.6054. R^2: 0.0681.
INFO:root:Epoch 600. Training loss: 6.5956. R^2: 0.0695.
INFO:root:Epoch 700. Training loss: 6.5878. R^2: 0.0706.
INFO:root:Epoch 800. Training loss: 6.5813. R^2: 0.0715.
INFO:root:Epoch 900. Training loss: 6.5750. R^2: 0.0724.
INFO:root:Epoch 1,000. Training loss: 6.5698. R^2: 0.0731.
INFO:root:Epoch 1,100. Training loss: 6.5646. R^2: 0.0738.
INFO:root:Epoch 1,200. Training loss: 6.5604. R^2: 0.0744.
INFO:root:Epoch 1,300. Training loss: 6.5563. R^2: 0.0750.
INFO:root:Epoch 1,400. Training loss: 6.5523. R^2: 0.0756.
INFO:root:Epoch 1,500. Training loss: 6.5487. R^2: 0.0761.
INFO:root:Epoch 1,600. Training loss: 6.5453. R^2: 0.0766.
INFO:root:Epoch 1,

In [35]:
loss_train = model_trainer2.model.loss(x_train, y_train)
r2_score_train = 1 - loss_train/jnp.var(y_train)

loss_test = model_trainer2.model.loss(x_test, y_test)
r2_score_test = 1 - loss_test/jnp.var(y_test)

logging.info(f'Train loss {loss_train:.3f}. R^2: {r2_score_train:.3f}.')
logging.info(f'Test loss {loss_test:.3f}. R^2: {r2_score_test:.3f}.')

INFO:root:Train loss 6.466. R^2: 0.088.
INFO:root:Test loss 6.102. R^2: 0.085.


In [42]:
with open('./__state__/symmetric_local_model_trainer.pkl', 'wb') as file:
    dill.dump(model_trainer2, file)

## Local Transformer Model

In [30]:
# TODO: Model architecture needs optimization to run in reasonable time
model3 = LocalTransformerEncoderModel()

In [31]:
model3(x_train[0])

Array([1.9228151], dtype=float32)

In [32]:
model3.loss(x_train, y_train)

Array(9.713807, dtype=float32)

In [33]:
model_trainer3 = ModelTrainer(model3, epochs=100, optimizer=optax.sgd)

In [47]:
model_trainer3.train(x_train, y_train)

INFO:root:Epoch 0. Training loss: 11.3904. R^2: -0.6109.


KeyboardInterrupt: 